In [3]:
import pandas as pd 
code_df = pd.read_html('http://kind.krx.co.kr/corpgeneral/corpList.do?method=download&searchType=13', header=0)[0] 
# 종목코드가 6자리이기 때문에 6자리를 맞춰주기 위해 설정해줌 
code_df.종목코드 = code_df.종목코드.map('{:06d}'.format) 
# 우리가 필요한 것은 회사명과 종목코드이기 때문에 필요없는 column들은 제외해준다. 
code_df = code_df[['회사명', '종목코드']] 
# 한글로된 컬럼명을 영어로 바꿔준다. 
code_df = code_df.rename(columns={'회사명': 'name', '종목코드': 'code'}) 
code_df.head()



,name,code
0,DSR,155660
1,GS글로벌,001250
2,HSD엔진,082740
3,LG이노텍,011070
4,LS산전,010120


In [7]:
# 종목 이름을 입력하면 종목에 해당하는 코드를 불러와 
# 네이버 금융(http://finance.naver.com)에 넣어줌 
def get_url(item_name, code_df): 
    code = code_df.query("name=='{}'".format(item_name))['code'].to_string(index=False) 
    code = code.strip()
    url = 'http://finance.naver.com/item/sise_day.nhn?code={code}'.format(code=code) 
    print("요청 URL = {}".format(url)) 
    return url 
# 신라젠의 일자데이터 url 가져오기 
item_name='신라젠' 
url = get_url(item_name, code_df) 
print(url)
# 일자 데이터를 담을 df라는 DataFrame 정의 
df = pd.DataFrame() 
# 1페이지에서 20페이지의 데이터만 가져오기 
for page in range(1, 21): 
    pg_url = '{url}&page={page}'.format(url=url, page=page) 
    df = df.append(pd.read_html(pg_url, header=0)[0], ignore_index=True) 
    # df.dropna()를 이용해 결측값 있는 행 제거 
    df = df.dropna() 
    # 상위 5개 데이터 확인하기 
df.head()



요청 URL = http://finance.naver.com/item/sise_day.nhn?code=215600
http://finance.naver.com/item/sise_day.nhn?code=215600


,날짜,종가,전일비,시가,고가,저가,거래량
0,2020.02.14,12400.0,450.0,12850.0,12950.0,12250.0,1967304.0
1,2020.02.13,12850.0,300.0,13250.0,13300.0,12800.0,1580393.0
2,2020.02.12,13150.0,50.0,13150.0,13300.0,12900.0,1393190.0
3,2020.02.11,13100.0,100.0,13250.0,13500.0,13050.0,1339485.0
4,2020.02.10,13200.0,150.0,13300.0,13450.0,13050.0,2144716.0
